In [120]:
import csv
import json


def tid(tree_id: int) -> str:
        """
        Generate TreeID from tree_id in database
        Since postgresql serial is 4*8 bit, if id is too big,
        alter table and return raw number.
        TID format: 'T00A100000'
            first letter, 'T'
            2-4, digit or capital letter, 0-9 and A-Z
            5-10, digit
        Args:
            tree_id: tree_id, postgresql serial number

        Returns:
            tid: str
        """
        max_n = min(36 ** 3 * 100_000, 2 ** (8 * 4 - 1))
        prefix = 'T'
        n = 1_000_00
        if tree_id >= max_n:
            return 'T' + str(tree_id)
        # 0-9 and A-Z
        base = 26 + 10

        a, b = divmod(tree_id, n)
        letters = ''
        while a > 0:
            a, digit = divmod(a, base)
            if digit < 10:
                letters = str(digit) + letters
            else:
                letters = chr(ord('A') + (digit - 10)) + letters
        return prefix + f'{letters:>03}' + f'{b:05d}'

In [121]:
# use sql to output table from treebase

In [122]:
"""psql
copy (
select to_json(t) from (
	select ncbi_names.name_txt as root, trees.study_id, study.doi,
	treefile.treefile_id, trees.tree_id, trees.tree_label, trees.tree_title, trees.tree_type, trees.tree_kind, 
	trees.root as root_id, trees.tree_quality, trees.upload_date, trees.tree_type_new
	from trees inner join ncbi_names on trees.root=ncbi_names.tax_id 
	inner join study on trees.study_id=study.study_id
	inner join treefile on trees.tree_id = treefile.tree_id
	where ncbi_names.name_class='scientific name'
) as t)
to 'r:\\test.json';
"""

"psql\ncopy (\nselect to_json(t) from (\n\tselect ncbi_names.name_txt as root, trees.study_id, study.doi,\n\ttreefile.treefile_id, trees.tree_id, trees.tree_label, trees.tree_title, trees.tree_type, trees.tree_kind, \n\ttrees.root as root_id, trees.tree_quality, trees.upload_date, trees.tree_type_new\n\tfrom trees inner join ncbi_names on trees.root=ncbi_names.tax_id \n\tinner join study on trees.study_id=study.study_id\n\tinner join treefile on trees.tree_id = treefile.tree_id\n\twhere ncbi_names.name_class='scientific name'\n) as t)\nto 'r:\\test.json';\n"

In [123]:
trees = list()
with open('r:\\test.json', 'r') as _:
    for line in _:
        # handle escape character from psql
        line2 = line.replace(r'\\', '\\')
        x = json.loads(line2)
        x['tree_file'] = tid(x['tree_id']) + '.nwk'
        trees.append(x)
print(trees[214:216])
json.dump(trees, open('r:\\tree.json', 'w'), indent=True)

[{'root': 'Gymnopilus pampeanus', 'study_id': 1960, 'doi': None, 'treefile_id': 5709, 'tree_id': 2928, 'tree_label': 'Fig. 2', 'tree_title': 'Tribe Erysipheae', 'tree_type': 'Single', 'tree_kind': 'Species Tree', 'root_id': 205671, 'tree_quality': 'Unrated', 'upload_date': None, 'tree_type_new': 'treebase', 'tree_file': 'T00002928.nwk'}, {'root': 'Aphodius sp. BLACKmx', 'study_id': 1966, 'doi': None, 'treefile_id': 5720, 'tree_id': 2942, 'tree_label': 'Supplementary 04', 'tree_title': 'Phialocephala, pPF-076', 'tree_type': 'Single', 'tree_kind': 'Species Tree', 'root_id': 207097, 'tree_quality': 'Unrated', 'upload_date': None, 'tree_type_new': 'treebase', 'tree_file': 'T00002942.nwk'}]


In [124]:
"""psql
copy (
	select to_json(t) from 
	  (select tree_id,newick from treefile) as t)
to 'r:\\treefile.json';
"""

"psql\ncopy (\n\tselect to_json(t) from \n\t  (select tree_id,newick from treefile) as t)\nto 'r:\\treefile.json';\n"

In [125]:
treefile = list()
with open('r://treefile.json', 'r') as _:
    for line in _:
        line2 = line.replace(r'\\', '\\')
        t = json.loads(line2)
        filename = tid(t['tree_id']) + '.nwk'
        with open(f'r://out/{filename}', 'w') as out_:
            out_.write(t['newick'])

In [126]:
"""psql
copy (
	select to_json(t) from (
select array_agg(treefile.tree_id) as tree_files, study.* from study
right join trees on study.study_id = trees.study_id
inner join treefile on trees.tree_id = treefile.tree_id
group by study.study_id
order by study.study_id) as t)
to 'r:\\paper.raw';
"""

"psql\ncopy (\n\tselect to_json(t) from (\nselect array_agg(treefile.tree_id) as tree_files, study.* from study\nright join trees on study.study_id = trees.study_id\ninner join treefile on trees.tree_id = treefile.tree_id\ngroup by study.study_id\norder by study.study_id) as t)\nto 'r:\\paper.raw';\n"

In [127]:
paper = list()
with open('r://paper.raw', 'r') as _:
    for line in _:
        line2 = line.replace(r'\\', '\\')
        t = json.loads(line2)
        t['tree_files'] = [tid(tf)+'.nwk' for tf in t['tree_files']]
        paper.append(t)
        
                           


In [128]:
print(paper[4:7])

[{'tree_files': ['T00001585.nwk'], 'study_id': 89, 'pub_type': 'A', 'author': 'Berry, P. E.', 'year': 1989, 'title': 'A systematic revision of Fuchsia sect. Quelusia (Onagraceae).', 'journal': 'Annals of the Missouri Botanical Garden', 's_author': None, 's_title': None, 'place_pub': None, 'publisher': None, 'volume': '76', 'number': None, 'pages': '532-584', 'isbn': None, 'keywords': None, 'abstract': 'Fuchsia sect. Quelusia consists of nine closely related species, eight from southeastern Brazil and one from southern Chile and Argentina. This treatment recognizes F. alpestris and F. glazioviana as separate from F. regia and describes two new species from southern Brazil, F. brevilobis and F. hatschbachii. The section is defined by its shrubby-lianoid habit, opposite-whorled leaves, and distinctive floral pattern associated with hummingbird pollination. It is one of only two entirely polyploid sections in the genus. Unique characters of the section include the large, violet, convolute 

In [129]:
with open(f'r://paper.json', 'w') as out_:
    json.dump(paper, out_, indent=True)

In [130]:
"""
1. import to database
2. output merged final json (paper.json,tree.json) and newick treefiles
3. statistics
"""


'\n1. import to database\n2. output merged final json (paper.json,tree.json) and newick treefiles\n3. statistics\n'

In [133]:
with open(f'r://paper.json') as _:
    data2 = json.load(_)
header = data2.pop()
header['tree_files'] = len(header['tree_files'])
print(header)
with open(r'r://for_draw.csv', 'w', newline='') as out:
    writer = csv.DictWriter(out, fieldnames=list(header.keys()))
    writer.writeheader()
    writer.writerow(header)
    for record in data2:
        record['tree_files'] = len(record['tree_files'])
        writer.writerow(record)



{'tree_files': 3, 'study_id': None, 'pub_type': None, 'author': None, 'year': None, 'title': None, 'journal': None, 's_author': None, 's_title': None, 'place_pub': None, 'publisher': None, 'volume': None, 'number': None, 'pages': None, 'isbn': None, 'keywords': None, 'abstract': None, 'legacy_id': None, 'url': None, 'doi': None, 'upload_date': None}
